IoT-DTIM
======

- This Notebook only contains the functions of device clustering module and deep-learning models.
    - It describes the device clustering method and its corresponding functions.
    - Also, it explains the structure of each deep learning model and what shape the input and output are.
- This code is written in an environment with python 3.6. Also, the version of TensorFlow and Keras are 1.12.0 and 2.2.5, respectively.

# Function for training of Device Clustering 

### Description
- This function trains the device clustering model and returns the trained autoencoder and encoder model.
- The input data is scaled time-series data with two modalities such as temperature and humidity.
- Shape of input data "X_train" and "X_vaild" is (:, window_size * 2). 
- The "X_train[ : , : window_size ]" is temperature data and The last of "X_train[ : , window_size : ]" is humidity data. 
- The "X_valid" is also same. 
- This function returns the trained autoencoder model and encoder model.
- The only encoder model will be used in device_clustering_module.

### 1. training_device_clustering()

- Input:
    - X_train : input and output of autoencoder model for train data in training phase
    - X_valid : input and output of autoencoder model for validation data in training phase
    - window_size : training window size, this paper set it to 120

- Output:
    - Rreturns trained autoencoder and encoder model

### Example
    autoencoder, encoder = training_of_device_clustering(X_train, X_valid, window_size)



In [1]:
def training_device_clustering(X_train, X_valid, window_size):
	encoding_dim1 = 128
	encoding_dim2 = 64
	encoding_dim3 = 64
	nb_epoch= 100
	batch_size = 32

	target_sensor = ['temp', 'humi']
	input_dim = window_size
	print('input dimension is ', input_dim )
	input_layer_1= Input (shape = (input_dim,))
	input_layer_2 = Input (shape = (input_dim,))

	en_L1_1 = Dense (encoding_dim1, activation = "relu")(input_layer_1)
	en_L1_2 = Dense (encoding_dim1, activation = "relu")(input_layer_2)
	en_L2_1 = Dense (encoding_dim2, activation = "relu")(en_L1_1)
	en_L2_2 = Dense (encoding_dim2, activation = "relu")(en_L1_2)
	merge = concatenate([en_L2_1, en_L2_2])
	merge = Dense (encoding_dim3, activation = 'relu')(merge)
	e_L3 = Dense (encoding_dim3, activation = 'relu')(merge)
	d_L2_1 = Dense (encoding_dim2, activation = "relu")(e_L3)
	d_L2_2 = Dense (encoding_dim2, activation = "relu")(e_L3)
	d_L1_1 = Dense (encoding_dim1, activation = "relu")(d_L2_1)
	d_L1_2 = Dense (encoding_dim1, activation = "relu")(d_L2_2)
	d_1_additional = Dense (input_dim, activation = "relu")(d_L1_1)
	d_2_additional = Dense (input_dim, activation = "relu")(d_L1_2)
	d_1 = Dense (input_dim, activation = 'linear' )(d_1_additional)
	d_2 = Dense (input_dim, activation = 'linear' )(d_2_additional)

	autoencoder = Model (inputs = [input_layer_1, input_layer_2], outputs = [d_1, d_2])
	encoder = Model (inputs = [input_layer_1, input_layer_2], outputs = e_L3)

	autoencoder.summary()
	encoder.summary()

	plot_model(autoencoder, to_file='plot_model/AE.png', show_shapes = True)
	autoencoder.compile(optimizer='adadelta', loss='mean_squared_error', metrics=['accuracy'])
	heckpointer = ModelCheckpoint(filepath="models/IoT_clustering_AE_model.h5", verbose=0, save_best_only=True)
	tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0, write_graph=True, write_images=True)
	history = autoencoder.fit([X_train[:,:window_size], X_train[:,window_size:]], [X_train[:,:window_size], X_train[:,window_size:]],
	                    epochs=nb_epoch,
	                    batch_size=batch_size,
	                    shuffle=True,
	                    validation_data=([X_valid[:,:window_size], X_valid[:,window_size:]], [X_valid[:,:window_size], X_valid[:,window_size:]]),
	                    verbose=2,
	                    callbacks=[checkpointer, tensorboard]).history
	plt.plot(history['loss'])
	plt.plot(history['val_loss'])
	plt.title('model loss')
	plt.ylabel('loss')
	plt.xlabel('epoch')
	plt.legend(['train', 'test'], loc='upper right');
	return autoencoder, encoder

# Device clustering based on trained encoder model and K-means clustering algorithm

### Description
- The purpose of this procedure is to label the each sensor based on trained encoder model using PCA and K-means clustering algorithm.
- First, the "make_feature_map()" function is used to extract the features from the time series data of each device through an encoder model.
- Next, the dimension of the feature is reduced by using PCA on the extracted features, and the K-means algorithm is applied.
- At this time, the "find_best_nb_of_clustering()" function selects the optimal number of clusters based on the elbow-point method.
- Labeling of each device is performed using a clustering algorithm based on the number of selected clusters.

## 1. make_feature_map()
- Input:
    - encoder : trained encoder model 
    - X_test_each_sensor_for_clustering : test data of encoder model 
        - This input is reshaped to the input shape of encoder model 

- Output:
    - feature_each_device : returns the encoded feature for test input data

## 2. find_best_nb_of_clustering()
- Input:
    - feature_each_device : extraced feature from make_feature_map() function 
    - test_data_index : target test data index

- Output:
    - best_k : returns the best number of clusters

    
## 3. clustering()
- Input:
    - feature_each_device : extraced feature from make_feature_map() function 
    - km_n_c : best number of clusters results from find_best_nb_of_clustering() function
    - test_data_index : target test data index

- Output:
    - device_clustering_dict : returns the devices cluster labeling results

### Example
    feature_each_device = make_feature_map(encoder, X_test_each_device_for_clustering)
    km_n_c = find_best_nb_of_clustering (feature_each_device, test_data_index)
    device_clustering_dict = clustering(feature_each_device, km_n_c, test_data_index)

    
    




In [2]:
##################################################################################################
##################################################################################################
def make_feature_map(encoder,X_test_each_sensor_for_clustering):
    input_data = X_test_each_sensor_for_clustering
    clusterig_window_size =  input_data['temp'].shape[1]
    nb_devices = input_data['temp'].shape[2]
    X_test_temp = np.transpose(input_data['temp'], (0,2,1)).reshape(-1, clusterig_window_size)
    X_test_humi = np.transpose(input_data['humi'], (0,2,1)).reshape(-1, clusterig_window_size)
    X_test_scaled = np.hstack([X_test_temp, X_test_humi])

    X_test_scaled_each_device = X_test_scaled.reshape(-1, nb_devices, clusterig_window_size * 2)
    feature_each_device = []
    for device_id in range(0, nb_devices):
        E_result = encoder.predict([X_test_scaled_each_device[:,device_id,:clusterig_window_size], X_test_scaled_each_device[:,device_id,clusterig_window_size:]])
        feature_each_device.append(E_result)


    feature_each_device = np.array(feature_each_device)
    return feature_each_device

##################################################################################################
##################################################################################################
def find_best_nb_of_clustering (feature_each_device, test_data_index):
    best_k = 0
    c_start = 2
    c_end = 8
    c_inertia = []
    c_sil_score = []
    feature = feature_each_device[:,test_data_index,:]

    for km_n_c in range(c_start,c_end):
        pca = PCA(n_components = 2)
        pca_result = pca.fit_transform(feature)
        kmeans_model = KMeans(n_clusters = km_n_c)
        inertia = kmeans_model.fit(pca_result).inertia_
        c_inertia.append(inertia)

    c_inertia = np.array(c_inertia)
    best_k = np.argmax(np.diff(c_inertia, 2)) + 1 + c_start

    return best_k

##################################################################################################
##################################################################################################
def clustering(feature_each_device, km_n_c, test_data_index):
    km_device_map = np.zeros((8,16)).tolist()
    device_clustering_dict={}
    for i in range(0, km_n_c):
        device_clustering_dict[i]=[]
    for i in range(0,8):
        for j in range(0,16):
            km_device_map[i][j]= ''

    km_labels = []
    data = feature_each_device[:,test_data_index,:]
    # KMEAN
    pca = PCA(n_components=2)
    pca_result = pca.fit_transform(data)
    km_labels.append(KMeans(n_clusters=km_n_c).fit_predict(pca_result))
    km_labels = np.array(km_labels)
    device_index=0
    for key, val in device_location_map.items():
        km_device_map[val[0]][val[1]] = str(km_labels[0, device_index])
        device_clustering_dict[km_labels[0, device_index]].append(device_index)

        device_index += 1
    print("-------------------------------------------------")
    print_device_map(km_device_map)
    print("-------------------------------------------------")
    print()
    
    return device_clustering_dict


# Function for Training of Global LSTM model

### Description
- This function trains the Global LSTM model and saves best-trained model in the "models" folder.

### 1. training_global_lstm_model()
- Input:
    - model_name : saved model name
    - predict_size : number of time series data you want to predict, this paper set it to 10
    - nb_epoch : number of epoch, this paper set it to 100
    - batch_size : batch size of model, this paper set it to 32
    - X_train : input for training data in training phase
        - Shape : (:, 1, input_size)
    - Y_train : output for training data in training phase
        - Shape : (:, 1, predict_size)
    - X_valid : input for validation data in training phase
        - Shape : (:, 1, input_size)
    - Y_valid : output for validation data in training phase
        - Shape : (:, 1, predict_size)

- Output:
    - Save best trained model to "models" folder where filename is model_name+"LSTM.h5"


### Example 
    training_global_lstm_model('temp_global_',predict_size, nb_epoch, batch_size, X_train_temp, Y_train_temp, X_valid_temp, Y_valid_temp)
    training_global_lstm_model('humi_global_',predict_size, nb_epoch, batch_size, X_train_humi, Y_train_humi, X_valid_humi, Y_valid_humi)


In [3]:
def training_global_lstm_model(model_name, predict_size, nb_epoch, batch_size, X_train, Y_train, X_valid, Y_valid):
    input_layer= Input (shape = (1, input_size))

    lstm_left_1 = LSTM(64, activation='relu', return_sequences=True)(input_layer)
    lstm_left_2 = LSTM(64, activation='relu', return_sequences=True)(lstm_left_1)
    # lstm_left_3 = LSTM(64, activation='relu', return_sequences=True)(lstm_left_2)
    output_left = TimeDistributed(Dense(predict_size))(lstm_left_2)
    lstm = Model (inputs = input_layer, outputs = output_left)

    lstm.summary()
    lstm.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
    plot_model(lstm, to_file='plot_model/lstm_model.png', show_shapes = True)

    checkpointer = ModelCheckpoint(filepath="models/"+model_name+"LSTM.h5", verbose=0, save_best_only=True)
    tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0, write_graph=True, write_images=True)
    history = lstm.fit(X_train, Y_train,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    shuffle=True,
                    validation_data=(X_valid, Y_valid),
                    verbose=1,
                    callbacks=[checkpointer, tensorboard]).history


# Function for Training of Local LSTM model

### Description
- This function fine-tunes of Global LSTM model using clustering results.
- This function receives the global_lstm model as an input argument and fix the weights of LSTM layers 1 and 2 so that they are not trained.
- After that, the local LSTM model of each cluster is trained by adding layers that can learn the characteristics of each cluster.
- Input and validation dataset are data of each cluster.

### 1. training_local_lstm_model()
- Input:
    - predict_size : number of time series data you want to predict, this paper set it to 10
    - nb_epoch_2 : epoch of Local LSTM model, this paper set it to 40
    - batch_size : batch size of model, this paper set it to 32
    - global_lstm : pretrained Global LSTM model
    - input_layer : input layer of pretrained Global LSTM model
    - output_left_1 : output layer of pretrained Global LSTM model
    - c_idx : clustering index (if the number of clusters is 4, then it can be 0, 1, 2, 3)
    - X_train_tl : input for training data in training phase
        - Shape : (:, 1, input_size)
    - Y_train_tl : output for training data in training phase
        - Shape : (:, 1, predict_size)
    - X_valid_tl : input for validation data in training phase
        - Shape : (:, 1, input_size)
    - Y_valid_tl : output for validation data in training phase
        - Shape : (:, 1, predict_size)

- Output
    - Returns trained Local LSTM model 


### Example 
    temp_global_lstm = load_model("models/temp_global_LSTM.h5", compile=False)
    humi_global_lstm = load_model("models/humi_global_LSTM.h5", compile=False)

    temp_global_lstm.layers[1].name = 'lstm_temp_global_1'
    temp_global_lstm.layers[2].name = 'lstm_temp_global_2'
    temp_global_lstm.layers[3].name = 'time_distributed_temp_global_1'

    humi_global_lstm.layers[1].name = 'lstm_humi_global_1'
    humi_global_lstm.layers[2].name = 'lstm_humi_global_2'
    humi_global_lstm.layers[3].name = 'time_distributed_humi_global_1'
    
    tl_model_temp = training_local_lstm_model(predict_size, nb_epoch_2, batch_size,
                    humi_global_lstm, humi_global_lstm.input, humi_global_lstm.output, c_idx,
                    X_train_tl, Y_train_tl, X_valid_tl, Y_valid_tl)
    tl_model_humi = training_local_lstm_model(predict_size, nb_epoch_2, batch_size,
                    humi_global_lstm, humi_global_lstm.input, humi_global_lstm.output, c_idx,
                    X_train_tl, Y_train_tl, X_valid_tl, Y_valid_tl)


In [4]:
def training_local_lstm_model(predict_size, nb_epoch_2, batch_size, global_lstm, input_layer, output_left_1, c_idx, X_train_tl, Y_train_tl, X_valid_tl, Y_valid_tl):
    global_lstm.layers[1].trainable = False
    global_lstm.layers[2].trainable = False

    lstm_right_1 = LSTM(64, activation='relu', return_sequences=True)(input_layer)
    lstm_right_2 = LSTM(64, activation='relu', return_sequences=True)(lstm_right_1)
    # output_left  = TimeDistributed(Dense(predict_size))(lstm_left_2)
    output_right = TimeDistributed(Dense(predict_size))(lstm_right_2)
    merged = concatenate([output_left_1, output_right])
    merged = Dense(32,activation = 'relu')(merged)
    output_layer = Dense(predict_size, activation = 'linear')(merged)

    tl_model = Model (inputs=input_layer, outputs=output_layer)

    opti = optimizers.Adam(lr=0.001)
    tl_model.compile(optimizer=opti, loss='mean_squared_error', metrics=['accuracy'])
    checkpointer = ModelCheckpoint(filepath="models/LSTM_final.h5", verbose=0, save_best_only=True)
    tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0, write_graph=False, write_images=False)

    history = tl_model.fit(X_train_tl[c_idx], Y_train_tl[c_idx],
                        epochs=nb_epoch_2,
                        batch_size=batch_size,
                        shuffle=True,
                        validation_data=(X_valid_tl[c_idx], Y_valid_tl[c_idx]),
                        verbose=0,
                        callbacks=[checkpointer, tensorboard]).history

    return tl_model


# The device interval management algorithm applies the trained Local LSTM model to each device.
- The transmission interval of each device is determined by predicting future data through the Local LSTM model and comparing it with the measured data.
- For detailed algorithm, please refer to the algorithm in the paper.